In [1]:
%load_ext autoreload
%autoreload 2

In [43]:
from DataLoader import MyDataloader

In [44]:
from pathlib import Path
import pandas as pd

In [51]:
path = Path(r"/mnt/Data1/Python_Projects/Pure-Python/P5/01-Thesis/data/Shiraze94_en")
data = [pd.read_csv(path/"activity_chain.csv"), pd.read_csv(path/"OD_matrix_area.csv"), pd.read_csv(path/"trips.csv")]
dataloader = MyDataloader(data_list=data, seed=42)

In [ ]:


print(dataloader.data_activity.columns)

Index(['person_number_in_family', 'activity_chain', 'time_chain',
       'purpose_chain', 'duration', 'area_zones', 'region_zones',
       'travel_mode', 'trip_distance', 'hourly_correction_factor', 'gender',
       'age', 'job', 'working', 'driving_license', 'education_level',
       'home_area_code', 'home_region_code', 'family_members_count',
       'total_vehicles_count', 'bicycles_24inch_or_larger_count',
       'motorcycles_count', 'private_cars_count', 'pickup_trucks_count',
       'taxis_count', 'other_vehicles_count', 'private_car_1', 'private_car_2',
       'private_car_3', 'private_car_4', 'correction_factor',
       'expansion_factor', 'ends_in_home'],
      dtype='object')


In [48]:
list_cols = ["activity_chain", 'time_chain', 'purpose_chain', 'area_zones', 'region_zones', "duration", "travel_mode"]

dataloader.data_activity

,person_number_in_family,activity_chain,time_chain,purpose_chain,duration,area_zones,region_zones,travel_mode,trip_distance,hourly_correction_factor,...,pickup_trucks_count,taxis_count,other_vehicles_count,private_car_1,private_car_2,private_car_3,private_car_4,correction_factor,expansion_factor,ends_in_home
0,1,"[120670, 120671]","[615, 870]","[visit_offices, return_home]",[255],"[252, 128, 252]","[14, 7, 14]","[private_car, private_car]","[8499.15, 7693.01]","[1.7, 1.7]",...,0,0,0,-,-,-,-,0.66181,29.56596,1.0
1,1,"[100368, 100370, 100371, 100374, 100380, 100386]","[450, 515, 757, 870, 925, 1063]","[recreation_or_pilgrimage, return_home, shoppi...","[65, 242, 113, 55, 138]","[171, 170, 171, 6, 13, 82, 171]","[14, 14, 14, 1, 1, 6, 14]","[taxi, taxi, city_bus, taxi, city_bus, taxi]","[2421.68, 1677.5, 7807.36, 1374.95, 3250.17, 5...","[1.0, 1.0, 1.0, 1.7, 1.0, 1.0]",...,0,0,0,-,-,-,-,0.76409,34.47413,1.0
2,2,"[100392, 100397]","[630, 830]","[shopping, return_home]",[200],"[171, 122, 171]","[14, 6, 14]","[city_bus, taxi]","[3683.34, 3047.36]","[1.0, 1.0]",...,0,0,0,-,-,-,-,0.76409,34.47413,1.0
3,3,"[100404, 100408]","[390, 850]","[education, return_home]",[460],"[171, 118, 171]","[14, 6, 14]","[taxi, taxi]","[3886.69, 3863.43]","[1.0, 1.0]",...,0,0,0,-,-,-,-,0.76409,34.47413,1.0
4,4,"[100412, 100413]","[390, 860]","[education, return_home]",[470],"[171, 122, 171]","[14, 6, 14]","[taxi, taxi]","[3683.34, 3047.36]","[1.0, 1.0]",...,0,0,0,-,-,-,-,0.76409,34.47413,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39553,1,"[107685, 107686, 107687, 107688]","[525, 705, 910, 1125]","[education, return_home, education, return_home]","[180, 205, 215]","[27, 30, 27, 30, 27]","[1, 1, 1, 1, 1]","[city_bus, city_bus, city_bus, city_bus]","[1268.87, 1141.41, 1268.87, 1141.41]","[1.0, 1.0, 1.0, 1.0]",...,0,0,0,-,-,-,-,1.00000,6.50000,1.0
39554,1,"[107708, 107709, 107710, 107711, 107712]","[585, 810, 850, 1080, 1160]","[education, dropoff_or_pickup_others, return_h...","[225, 40, 230, 80]","[27, 30, 38, 27, 34, 27]","[1, 1, 1, 1, 1, 1]","[private_car, private_car, private_car, privat...","[1268.87, 4128.48, 4348.42, 2497.62, 2238.56]","[1.0, 1.0, 1.0, 1.0, 1.0]",...,0,0,0,پراید - سال 1391,-,-,-,1.00000,6.50000,1.0
39555,1,"[107680, 107681, 107682, 107683, 107684]","[450, 585, 825, 1020, 1185]","[medical_purposes, education, return_home, edu...","[135, 240, 195, 165]","[27, 105, 30, 27, 30, 27]","[1, 4, 1, 1, 1, 1]","[city_bus, city_bus, city_bus, taxi, city_bus]","[3620.15, 2683.35, 1141.41, 1268.87, 1141.41]","[1.0, 1.0, 1.0, 1.0, 1.0]",...,0,0,0,-,-,-,-,1.00000,6.50000,1.0
39556,1,"[107676, 107677, 107678, 107679]","[630, 705, 910, 1110]","[shopping, return_home, education, return_home]","[75, 205, 200]","[27, 6, 27, 30, 27]","[1, 1, 1, 1, 1]","[bicycle, bicycle, bicycle, bicycle]","[2449.41, 1543.02, 1268.87, 1141.41]","[1.0, 1.0, 1.0, 1.0]",...,0,0,0,-,-,-,-,1.00000,6.50000,1.0
